In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [2]:
# Read files
def read_data(file_name):
    df = pd.read_csv(file_name, comment = ';')
    df["time"] = pd.date_range(pd.to_datetime('2021-04-14'+' '+df.time[0]),periods = df.shape[0], freq='0.01086S')
    df.set_index('time', inplace = True)
    df.index = df.index.values.astype('M8[ms]')
    df = df.resample("10L", base = 4).first().interpolate()
    df = df.rename(columns = {"index":"time"})
    return df
def read_esense(file):
    df = pd.read_csv(file,header=None)
    df.columns = ['time','device','accx','accy','accz','acc1x','acc1y','acc1z','label']
    df['time'] = pd.to_datetime(df['time'], unit='ms') + pd.Timedelta(hours = 2)
    df.index = df['time']
    df = df.resample("10L", base = 4).first().interpolate().drop(columns = ["time","device","label"])
    return df   

In [3]:
# For subject2 ankle
dribble_sub2_ankle = read_data('dribbling_sbj2_ankle.csv')
layup_sub2_ankle = read_data('layup_sbj2_ankle.csv')
movements_sub2_ankle_1 = read_data('movements_sbj2_ankle_1.csv')
shooting_sub2_ankle_1 = read_data('shooting_sbj2_ankle_1.csv')
df_sub2_ankle = pd.concat([shooting_sub2_ankle_1,layup_sub2_ankle,dribble_sub2_ankle,movements_sub2_ankle_1])
df_sub2_ankle

,acc_x,acc_y,acc_z
2021-04-14 18:55:50.354,-0.354000,0.433840,-0.84033
2021-04-14 18:55:50.364,-0.343020,0.437010,-0.85181
2021-04-14 18:55:50.374,-0.343990,0.431400,-0.85742
2021-04-14 18:55:50.384,-0.345950,0.432620,-0.86060
2021-04-14 18:55:50.394,-0.346680,0.432865,-0.85779
...,...,...,...
2021-04-14 19:43:01.824,-0.980470,-0.197020,-0.04956
2021-04-14 19:43:01.834,-0.994630,-0.196040,-0.06299
2021-04-14 19:43:01.844,-1.001710,-0.172610,-0.08545
2021-04-14 19:43:01.854,-0.994995,-0.168825,-0.08777


In [4]:
#For subject1 ankle
layup_sub1_ankle = read_data('layup_sbj1_ankle.csv')
movements_sub1_ankle_1 = read_data('movements_sbj1_ankle_1.csv')
shooting_sub1_ankle = read_data('shooting_sbj1_ankle.csv')
df_sub1_ankle = pd.concat([shooting_sub1_ankle,layup_sub1_ankle,movements_sub1_ankle_1])
df_sub1_ankle

,acc_x,acc_y,acc_z
2021-04-14 18:12:54.024,0.232180,-0.959720,-0.08081
2021-04-14 18:12:54.034,0.231200,-0.960450,-0.08350
2021-04-14 18:12:54.044,0.233400,-0.961670,-0.08325
2021-04-14 18:12:54.054,0.232420,-0.964840,-0.08496
2021-04-14 18:12:54.064,0.234620,-0.960940,-0.08643
...,...,...,...
2021-04-14 18:50:00.944,-0.021725,-0.699955,-0.70557
2021-04-14 18:50:00.954,-0.021970,-0.701420,-0.70557
2021-04-14 18:50:00.964,-0.020750,-0.696780,-0.70532
2021-04-14 18:50:00.974,-0.021730,-0.697020,-0.70679


In [5]:
# For esense subject 2
df_esense_sub2 = read_esense('eSense_sbj2.csv')
df_esense_sub2

,accx,accy,accz,acc1x,acc1y,acc1z
time,,,,,,
2021-04-14 19:00:07.664,-0.875000,-0.154053,0.011719,-7.835366,7.591463,-3.079268
2021-04-14 19:00:07.674,-0.875464,-0.153760,0.010693,-8.085366,7.533537,-2.917683
2021-04-14 19:00:07.684,-0.875928,-0.153467,0.009668,-8.335366,7.475610,-2.756098
2021-04-14 19:00:07.694,-0.876392,-0.153174,0.008643,-8.585366,7.417683,-2.594512
2021-04-14 19:00:07.704,-0.876855,-0.152881,0.007617,-8.835366,7.359756,-2.432927
...,...,...,...,...,...,...
2021-04-14 19:42:09.154,-0.794336,0.251855,-0.612500,-1.304878,4.603659,-2.603659
2021-04-14 19:42:09.164,-0.790576,0.254810,-0.608911,-0.338415,5.137195,-3.713415
2021-04-14 19:42:09.174,-0.786816,0.257764,-0.605322,0.628049,5.670732,-4.823171


In [6]:
# For esense subject 1
df_esense_sub1 = read_esense('eSense_sbj1.csv')
df_esense_sub1

,accx,accy,accz,acc1x,acc1y,acc1z
time,,,,,,
2021-04-14 18:11:47.364,-0.461182,0.815430,-0.323242,-3.932927,1.737805,4.451220
2021-04-14 18:11:47.374,-0.464844,0.824707,-0.324951,-3.079268,1.554878,2.743902
2021-04-14 18:11:47.384,-0.465088,0.824487,-0.324219,-3.030488,1.310976,2.673780
2021-04-14 18:11:47.394,-0.465332,0.824268,-0.323486,-2.981707,1.067073,2.603659
2021-04-14 18:11:47.404,-0.465576,0.824048,-0.322754,-2.932927,0.823171,2.533537
...,...,...,...,...,...,...
2021-04-14 18:49:10.754,-0.382965,-0.145905,0.936707,-1.379573,1.124238,0.038110
2021-04-14 18:49:10.764,-0.383057,-0.145996,0.936279,-1.371951,1.067073,0.030488
2021-04-14 18:49:10.774,-0.383545,-0.145264,0.936157,-1.204268,1.128049,0.015244


In [7]:
# For subject2 wrist
dribble_sub2_wrist = read_data('dribbling_sbj2_wrist.csv')
layup_sub2_wrist = read_data('layup_sbj2_wrist.csv')
movements_sub2_wrist = read_data('movements_sbj2_wrist.csv')
shooting_sub2_wrist = read_data('shooting_sbj2_wrist.csv')
df_sub2_wrist = pd.concat([shooting_sub2_wrist,layup_sub2_wrist,dribble_sub2_wrist,movements_sub2_wrist])
df_sub2_wrist

,acc_x,acc_y,acc_z
2021-04-14 18:56:21.464,0.20972,-0.04395,-0.18628
2021-04-14 18:56:21.474,0.16040,-0.00146,-0.15186
2021-04-14 18:56:21.484,0.07617,0.11768,-0.09424
2021-04-14 18:56:21.494,0.03687,0.15186,0.03882
2021-04-14 18:56:21.504,-0.00098,0.16260,0.18750
...,...,...,...
2021-04-14 19:42:17.924,0.83032,0.40356,-0.51953
2021-04-14 19:42:17.934,0.83960,0.40869,-0.53882
2021-04-14 19:42:17.944,0.84497,0.39966,-0.53345
2021-04-14 19:42:17.954,0.82813,0.37451,-0.46191


In [8]:
#For subject1 wrist
layup_sub1_wrist = read_data('layup_sbj1_wrist.csv')
movements_sub1_wrist = read_data('movements_sbj1_wrist.csv')
shooting_sub1_wrist = read_data('shooting_sbj1_wrist.csv')
df_sub1_wrist = pd.concat([shooting_sub1_wrist,layup_sub1_wrist,movements_sub1_wrist])
df_sub1_wrist

,acc_x,acc_y,acc_z
2021-04-14 18:13:12.824,0.93042,0.475830,0.108890
2021-04-14 18:13:12.834,0.94653,0.362300,0.057130
2021-04-14 18:13:12.844,0.98462,0.263915,0.030640
2021-04-14 18:13:12.854,1.02271,0.165530,0.004150
2021-04-14 18:13:12.864,1.01880,0.084230,-0.064940
...,...,...,...
2021-04-14 18:47:41.634,0.32983,0.773440,-0.415280
2021-04-14 18:47:41.644,0.33105,0.770510,-0.390870
2021-04-14 18:47:41.654,0.33618,0.777225,-0.381835
2021-04-14 18:47:41.664,0.34131,0.783940,-0.372800


In [9]:
# data for subject 2
df_sub2 = pd.merge(pd.merge(df_esense_sub2,df_sub2_ankle,left_index = True,right_index = True),df_sub2_wrist,left_index = True, right_index = True)
df_sub2 = df_sub2.rename(columns = {'acc_x_x':'ankle_x','acc_y_x':'ankle_y','acc_z_x':'ankle_z','accx':'esense_x','accy':'esense_y','accz':'esense_z','acc1x':'gyro_x','acc1y':'gyro_y','acc1z':'gyro_z','acc_x_y':'wrist_x','acc_y_y':'wrist_y','acc_z_y':'wrist_z'})
df_sub2

,esense_x,esense_y,esense_z,gyro_x,gyro_y,gyro_z,ankle_x,ankle_y,ankle_z,wrist_x,wrist_y,wrist_z
2021-04-14 19:00:07.664,-0.875000,-0.154053,0.011719,-7.835366,7.591463,-3.079268,-1.004880,0.02466,0.003420,0.394530,-0.089600,0.79492
2021-04-14 19:00:07.674,-0.875464,-0.153760,0.010693,-8.085366,7.533537,-2.917683,-1.007570,0.02661,-0.001460,0.414790,-0.112060,0.80835
2021-04-14 19:00:07.684,-0.875928,-0.153467,0.009668,-8.335366,7.475610,-2.756098,-1.005620,0.03101,0.000980,0.434445,-0.131835,0.82251
2021-04-14 19:00:07.694,-0.876392,-0.153174,0.008643,-8.585366,7.417683,-2.594512,-1.006100,0.02563,0.008060,0.454100,-0.151610,0.83667
2021-04-14 19:00:07.704,-0.876855,-0.152881,0.007617,-8.835366,7.359756,-2.432927,-1.005130,0.02124,0.004390,0.494140,-0.172360,0.87207
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-14 19:42:09.154,-0.794336,0.251855,-0.612500,-1.304878,4.603659,-2.603659,-0.873050,-0.42090,-0.217040,0.287600,0.893070,0.37256
2021-04-14 19:42:09.164,-0.790576,0.254810,-0.608911,-0.338415,5.137195,-3.713415,-0.874510,-0.41138,-0.222900,0.283450,0.888430,0.37158
2021-04-14 19:42:09.174,-0.786816,0.257764,-0.605322,0.628049,5.670732,-4.823171,-0.872310,-0.41138,-0.223140,0.286380,0.884520,0.36816
2021-04-14 19:42:09.184,-0.783057,0.260718,-0.601733,1.594512,6.204268,-5.932927,-0.868900,-0.41479,-0.222660,0.286130,0.880130,0.36670


In [10]:
# data for subject 1
df_sub1 = pd.merge(pd.merge(df_esense_sub1,df_sub1_ankle,left_index = True,right_index = True),df_sub1_wrist,left_index = True,right_index = True)
df_sub1 = df_sub1.rename(columns = {'acc_x_x':'ankle_x','acc_y_x':'ankle_y','acc_z_x':'ankle_z','accx':'esense_x','accy':'esense_y','accz':'esense_z','acc1x':'gyro_x','acc1y':'gyro_y','acc1z':'gyro_z','acc_x_y':'wrist_x','acc_y_y':'wrist_y','acc_z_y':'wrist_z'})
df_sub1

,esense_x,esense_y,esense_z,gyro_x,gyro_y,gyro_z,ankle_x,ankle_y,ankle_z,wrist_x,wrist_y,wrist_z
2021-04-14 18:13:12.824,-0.621021,0.654321,-0.168628,-56.750000,3.454268,14.978659,-0.98438,0.143800,0.13672,0.93042,0.475830,0.108890
2021-04-14 18:13:12.834,-0.621582,0.654785,-0.159912,-54.420732,3.109756,15.853659,-0.97852,0.132810,0.14478,0.94653,0.362300,0.057130
2021-04-14 18:13:12.844,-0.621737,0.657693,-0.153609,-53.051552,2.973947,16.424612,-0.99829,0.079100,0.17041,0.98462,0.263915,0.030640
2021-04-14 18:13:12.854,-0.621893,0.660600,-0.147306,-51.682373,2.838137,16.995565,-1.02441,0.032960,0.23364,1.02271,0.165530,0.004150
2021-04-14 18:13:12.864,-0.622048,0.663508,-0.141002,-50.313193,2.702328,17.566519,-1.01465,0.058840,0.27393,1.01880,0.084230,-0.064940
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-14 18:47:41.634,-0.625244,0.716309,-0.350342,-18.871951,4.329268,0.121951,0.00073,-0.606200,-0.76831,0.32983,0.773440,-0.415280
2021-04-14 18:47:41.644,-0.639160,0.692139,-0.334473,-20.091463,2.103659,-4.969512,0.01123,-0.617190,-0.75732,0.33105,0.770510,-0.390870
2021-04-14 18:47:41.654,-0.639313,0.688965,-0.331085,-19.599848,1.543445,-5.453506,0.00122,-0.615845,-0.77551,0.33618,0.777225,-0.381835
2021-04-14 18:47:41.664,-0.639465,0.685791,-0.327698,-19.108232,0.983232,-5.937500,-0.00879,-0.614500,-0.79370,0.34131,0.783940,-0.372800


In [11]:
# Labelling
column = ['esense_x','esense_y','esense_z','gyro_x','gyro_y','gyro_z','ankle_x','ankle_y','ankle_z','wrist_x','wrist_y','wrist_z']
df_sub2[column] = (df_sub2[column]-df_sub2[column].mean())/df_sub2[column].std()
df_sub1[column] = (df_sub1[column]-df_sub1[column].mean())/df_sub1[column].std()
time1 = [df_sub1.index.isin(df_sub1.between_time('18:33:28', '18:35:44').index), 
        df_sub1.index.isin(df_sub1.between_time('18:23:24', '18:25:21').index),
        df_sub1.index.isin(df_sub1.between_time('18:45:18', '18:47:10').index),
        df_sub1.index.isin(df_sub1.between_time('18:13:32', '18:15:40').index),
        df_sub1.index.isin(df_sub1.between_time('18:42:49', '18:45:17').index)]
subject_1 = ['dribbling', 'layup', 'running', 'shooting', 'walking']
df_sub1['Label'] = np.select(time1, subject_1, 'Null')
df_sub1 = df_sub1[['esense_x','esense_y','esense_z','gyro_x','gyro_y','gyro_z','ankle_x','ankle_y','ankle_z','wrist_x','wrist_y','wrist_z','Label']]

time2 = [df_sub2.index.isin(df_sub2.between_time('19:27:02', '19:29:55').index), 
        df_sub2.index.isin(df_sub2.between_time('19:17:43', '19:20:07').index),
        df_sub2.index.isin(df_sub2.between_time('19:39:46', '19:41:50').index),
        df_sub2.index.isin(df_sub2.between_time('19:00:20', '19:02:03').index),
        df_sub2.index.isin(df_sub2.between_time('19:37:19', '19:39:46').index)]
subject_2 = ['dribbling', 'layup', 'running', 'shooting', 'walking']
df_sub2['Label'] = np.select(time2, subject_2, 'Null')
df_sub2 = df_sub2[['esense_x','esense_y','esense_z','gyro_x','gyro_y','gyro_z','ankle_x','ankle_y','ankle_z','wrist_x','wrist_y','wrist_z','Label']]
df = pd.concat([df_sub1,df_sub2]).reset_index(drop=True)

In [13]:
# Label Encoding
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
label = le.fit_transform(df["Label"])
df["Label"] = label
df

,esense_x,esense_y,esense_z,gyro_x,gyro_y,gyro_z,ankle_x,ankle_y,ankle_z,wrist_x,wrist_y,wrist_z,Label
0,0.618585,1.694999,-1.013060,-0.825088,0.445602,0.569520,0.106162,0.390289,0.489337,0.672115,-0.175166,0.435134,0
1,0.617038,1.696019,-0.981509,-0.773401,0.431060,0.604190,0.113678,0.378607,0.506221,0.696044,-0.323477,0.358199,0
2,0.616610,1.702408,-0.958691,-0.743019,0.425328,0.626812,0.088322,0.321515,0.559910,0.752621,-0.452003,0.318825,0
3,0.616182,1.708798,-0.935873,-0.712636,0.419595,0.649435,0.054821,0.272469,0.692364,0.809198,-0.580529,0.279451,0
4,0.615754,1.715188,-0.913055,-0.682254,0.413863,0.672058,0.067339,0.299979,0.776764,0.803390,-0.686736,0.176758,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,0.185555,1.095468,-1.415892,0.222320,0.088492,0.065542,0.661651,0.643317,-0.191891,0.314562,0.304830,-0.377005,0
846,0.177351,1.090585,-1.415450,0.211902,0.094412,0.047059,0.622828,0.480077,-0.163757,0.419353,0.312812,-0.395510,0
847,0.169146,1.085703,-1.415008,0.201483,0.100331,0.028577,0.541719,0.303093,-0.120290,0.509291,0.305156,-0.428166,0
848,0.160942,1.080820,-1.414566,0.191065,0.106251,0.010094,0.418350,0.207586,-0.143291,0.552626,0.293510,-0.442866,0


In [14]:
# Splitting X and Y data
y_data = df['Label']
x_data = df.drop('Label',axis=1)

(163195, 12)


In [15]:
# Sliding window technique
def sliding_window(dataset,window_length,overlap_ratio):
    #Create empty lists
    windows = []
    indices = []
    non_overlap_elements = 0
    if overlap_ratio != None:
        overlap_elements = int((overlap_ratio/100)*(window_length))
    if overlap_elements >= window_length:
        print("Overlapping elements are more")
    while(non_overlap_elements < dataset.shape[0] - window_length):
        windows.append(dataset.iloc[non_overlap_elements:non_overlap_elements+window_length])
        indices.append([non_overlap_elements,non_overlap_elements+window_length])
        #Update non_overlap_elements
        non_overlap_elements = non_overlap_elements + window_length - overlap_elements
        try:
            final_windows = np.array(windows)
            final_indices = np.array(indices)
        except:
            final_windows = np.empty( shape =(len(windows), window_length, dataset.shape[1]), dtype =object)
            final_indices = np.array(indices)
            for i in range(0,len(windows)):
                final_windows[i] = windows[i]
                final_indices[i] = indices[i]
    return final_windows

In [16]:
window_length = 100
s_x = sliding_window(x_data,window_length,50)
s_y = sliding_window(y_data,window_length,50)
print(s_x.shape)
print(s_y.shape)

s_x = s_x.reshape(s_x.shape[0]*s_x.shape[1],s_x.shape[2])
s_y = s_y.reshape(s_y.shape[0]*s_y.shape[1],)
print(s_x.shape, s_y.shape)

(3262, 100, 12)
(3262, 100)
(326200, 12) (326200,)


In [17]:
import sklearn 
import keras.backend
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
kf = sklearn.model_selection.KFold(n_splits=6, shuffle=True, random_state=None)
print(kf)
accuracy = []
precision = []
recall = []
f1score = []
for train_index, test_index in kf.split(s_x):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = s_x[train_index], s_x[test_index]
    y_train, y_test = s_y[train_index], s_y[test_index]
    print("X train: ",X_train)
    print("Y train: ",y_train)
    print("X test: ",X_test)
    print("Y test: ",y_test)
    clf=RandomForestClassifier(n_estimators=100)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    pred_train = clf.predict(X_train)
    accuracy.append(metrics.accuracy_score(y_test, y_pred))
    precision.append(metrics.precision_score(y_test, y_pred, average = 'macro'))
    recall.append(metrics.recall_score(y_test, y_pred,average = 'macro'))
    f1score.append(metrics.f1_score(y_test, y_pred,average = 'macro'))
    keras.backend.clear_session()

Using TensorFlow backend.
C:\Users\saita\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\saita\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\saita\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\saita\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519

KFold(n_splits=6, random_state=None, shuffle=True)
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [18]:
print(np.mean(accuracy))
print(np.mean(precision))
print(np.mean(recall))
print(np.mean(f1score))

0.9968117736480577
0.9964998512419262
0.9966929567862776
0.9965951736429522
